In [1]:
!gsutil cp gs://cs327e-open-access-2/postgres_shopify.zip .

Copying gs://cs327e-open-access-2/postgres_shopify.zip...
/ [1 files][  3.2 MiB/  3.2 MiB]                                                
Operation completed over 1 objects/3.2 MiB.                                      


In [ ]:
!unzip postgres_shopify.zip

Archive:  postgres_shopify.zip
replace postgres_shopify/apps.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
import pandas as pd
from google.cloud import firestore
db = firestore.Client()
batch = db.batch()

In [ ]:
df1 = pd.read_csv('postgres_shopify/apps_categories.csv')

df2 = pd.read_csv('postgres_shopify/categories.csv')

combined_df = pd.merge(df1, df2, left_on = 'category_id', right_on = 'id')

combined_df.to_csv('combined.csv', index = False)

In [ ]:
batch = db.batch()

df = pd.read_csv('postgres_shopify/key_benefits.csv', sep = ',', header = 0, lineterminator = '\n')
key_benefits_rows = df.values.tolist()

df = pd.read_csv('postgres_shopify/apps.csv', sep = ',', header = 0, lineterminator = '\n')
apps_rows = df.values.tolist()

for key_benefits_row in key_benefits_rows:
    key_benefits_record = {}
    key_benefits_record['app_id'] = key_benefits_row[0]
    key_benefits_record['title'] = key_benefits_row[1]
    key_benefits_record['description'] = key_benefits_row[2]
    
    key_benefits_ref = db.collection('key_benefits').document(key_benefits_row[0])
    
    batch.set(key_benefits_ref, key_benefits_record)
    
    for row in apps_rows:
        if key_benefits_row[0] == apps_row[0]:
            apps_record = {}
            apps_record['id'] = apps_row[0]
            apps_record['url'] = apps_row[1]
            apps_record['title'] = apps_row[2]
            apps_record['developer'] = apps_row[3]
            apps_record['developer_link'] = apps_row[4]
            apps_record['icon'] = apps_row[5]
            apps_record['rating'] = apps_row[6]
            apps_record['review_count'] = apps_row[7]
            
            apps_ref = key_benefits_ref.collection('APPs').document(apps_row[0])
            
            batch.set(apps_ref, apps_record)
    batch.commit()

In [ ]:
batch = db.batch()

df = pd.read_csv('postgres_shopify/pricing_plans.csv', sep = ',', header = 0, lineterminator = '\n')
pp_rows = df.values.tolist()

df = pd.read_csv('postgres_shopify/pricing_plan_features.csv', sep = ',', header = 0, lineterminator = '\n')
pf_rows = df.values.tolist()

for row in pricing_rows:
    pp_record = {}
    pp_record['id'] = row[0]
    pp_record['app_id'] = row[1]
    pp_record['title'] = row[2]
    pp_record['price'] = row[3]
    
    pp_ref = db.collection('Pricing_plan_details').document(pp_rows[0])
    batch.set(pp_ref, pp_record)
    
    for row in feature_rows:
        if pp_rows[1] == row[0]:
            pf_record = {}
            pf_record['app_id'] = row[0]
            pf_record['pricing_plan_id'] = row[1]
            pf_record['feature'] = row[2]
            
            pf_ref = pp_ref.collection('pricing_plan_feature').document(pf_row[1])
            batch.set(pf_ref, pf_record)
    batch.commit()

In [ ]:
batch = db.batch()

df = pd.read_csv('combined.csv', sep = ',', header = 0, lineterminator = '\n')
c_rows = df.values.tolist()

df = pd.read_csv('postgres_shopify/apps.csv', sep = ',', header = 0, lineterminator = '\n')
apps_rows = df.values.tolist()

for rows in c_rows:
    c_record = {}
    c_record['app_id'] = rows[0]
    c_record['category_id'] = rows[1]
    c_record['title'] = rows[3]
    
    c_ref = db.collection('category').document(rows[1])
    batch.set(c_ref, c_record)
    for row in apps_rows:
        if rows[0] == row[0]:
            apps_record = {}
            apps_record['id'] = row[0]
            apps_record['url'] = row[1]
            apps_record['title'] = row[2]
            apps_record['developer'] = row[3]
            apps_record['developer_link'] = row[4]
            apps_record['icon'] = row[5]
            apps_record['rating'] = row[6]
            apps_record['review_count'] = row[7]
            
            apps_ref = c_ref.collection('apps').document(apps_row[0])
            

In [ ]:
categories_ref = db.collection('category')
query = categories_ref.where('title', '==', 'Productivity')
docs = query.stream()
for doc in docs:
    query = categories_ref.document(doc.id).collection('apps').order_by('rating', direction = firestore.Query.DESCENDING).limit(10)
    nresults = query.stream()
    
    for result in nresults:
        app_data = result.to_dict()
        app_id = result.id
        app_title = app_data['title']
        app_developer = app_data['developer']
        app_rating = app_data['rating']
        app_reviews_count = app_data['review_count']
        print(
            f"ID: {app_id}\nTitle: {app_title}\nDeveloper: {app_developer}\nRating: {app_rating}\nReviews Count: {app_reviews_count}\n"
        )

In [ ]:
apps_ref = db.collection('apps')
query = apps_ref.order_by('review_count', direction = firestore.Query.DESCENDING).limit(10)
docs = query.stream()
for doc in docs:
    app_data = doc.to_dict()
    app_id = doc.id
    app_title = app_data['title']
    app_developer = app_data['developer']
    app_rating = app_data['rating']
    app_reviews_count = app_data['review_count']
    print(
        f"ID: {app_id}\nTitle: {app_title}\nDeveloper: {app_developer}\nRating: {app_rating}\nReviews Count: {app_reviews_count}\n"
    )